In [ ]:
#Dev Babbar 20BAI10119
from google.colab import files
files.upload()

In [2]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [3]:
!kaggle datasets download -d akash2907/bird-species-classification

100% 1.37G/1.37G [01:09<00:00, 19.4MB/s]
100% 1.37G/1.37G [01:09<00:00, 21.3MB/s]


In [4]:
!mkdir Dataset
!cp /content/bird-species-classification.zip
!unzip -q /content/bird-species-classification.zip -d /content/Dataset
!rm /content/bird-species-classification.zip

cp: missing destination file operand after '/content/bird-species-classification.zip'
Try 'cp --help' for more information.


In [5]:
#Importing Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

In [6]:
#Directories
train_data_dir = 'Dataset/train_data/train_data'
test_data_dir = 'Dataset/test_data/test_data'

In [7]:
#Data Augmentation and rescaling
img_height, img_width = 224, 224
batch_size = 32
train_data_generator = ImageDataGenerator(rescale=1./255,  # normalize pixel values between 0 and 1
                                          rotation_range=20,  # random rotations
                                          width_shift_range=0.2,  # random horizontal shifts
                                          height_shift_range=0.2,  # random vertical shifts
                                          shear_range=0.2,  # random shearing
                                          zoom_range=0.2,  # random zoom
                                          horizontal_flip=True)  # random horizontal flips

test_data_generator = ImageDataGenerator(rescale=1./255)  # only rescale test data

train_generator = train_data_generator.flow_from_directory(train_data_dir,
                                                          target_size=(img_height, img_width),
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

test_generator = test_data_generator.flow_from_directory(test_data_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [8]:
#Transfer Learning using VGG16
model2 = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

58889256/58889256 [==============================] - 4s 0us/step


In [9]:
#Setting VGG16 layers to non-trainable
for layer in model2.layers:
    layer.trainable = False

In [10]:
#Adding VGG16 and our new layers
model = tf.keras.Sequential()
model.add(model2)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(train_generator.num_classes, activation='softmax'))

In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
epochs = 10
model.fit(train_generator,
          epochs=epochs,
          validation_data=test_generator)

Epoch 1/10
5/5 [==============================] - 88s 21s/step - loss: 0.8599 - accuracy: 0.7067 - val_loss: 3.1097 - val_accuracy: 0.3503
Epoch 2/10
5/5 [==============================] - 88s 20s/step - loss: 0.7459 - accuracy: 0.7600 - val_loss: 2.8863 - val_accuracy: 0.3758
Epoch 3/10
5/5 [==============================] - 87s 19s/step - loss: 0.4748 - accuracy: 0.8467 - val_loss: 3.1950 - val_accuracy: 0.3248
Epoch 4/10
5/5 [==============================] - 88s 20s/step - loss: 0.4940 - accuracy: 0.8067 - val_loss: 3.2429 - val_accuracy: 0.3631
Epoch 5/10
5/5 [==============================] - 87s 21s/step - loss: 0.3937 - accuracy: 0.8800 - val_loss: 3.1927 - val_accuracy: 0.3758
Epoch 6/10
5/5 [==============================] - 90s 20s/step - loss: 0.3903 - accuracy: 0.8800 - val_loss: 3.3480 - val_accuracy: 0.4140
Epoch 7/10
5/5 [==============================] - 96s 22s/step - loss: 0.3906 - accuracy: 0.8600 - val_loss: 3.3376 - val_accuracy: 0.3503
Epoch 8/10
5/5 [===========

In [15]:
model.evaluate(test_generator)

5/5 [==============================] - 49s 10s/step - loss: 3.2689 - accuracy: 0.3822


[3.268937587738037, 0.3821656107902527]